In [27]:
import re
import spacy
import TextBlob

voxUD = voxUsers[["user_name","user_description"]]
ppUD = ppUsers[["user_name","user_description"]]
csUD = csUsers[["user_name","user_description"]]
psoeUD = psoeUsers[["user_name","user_description"]]
upUD = upUsers[["user_name","user_description"]]

def removeStopwords(texto):
    texto = texto.lower()
    texto = re.sub('_', '', texto)
    texto = re.sub('@', '', texto)
    texto = re.sub(':', '', texto)
    texto = re.sub('/', '', texto)
    texto = re.sub('!', '', texto)
    texto = re.sub('_', '', texto)
    texto = re.sub('\.', '', texto)
    texto = deEmojify(texto)
    blob = TextBlob(texto).words
    outputlist = [word for word in blob if word not in stopwords.words('spanish')]
    return(outputlist)

def normalizeString(l):
    l = l.fillna('').values.tolist()
    for user in l:
        user[0] = removeStopwords(user[0])
        user[1] = removeStopwords(user[1]) 
    return l

def wordDict(l):
    dt = {}
    for sublist in l:
        for item in sublist:
            for item2 in item:
                if item2 not in dt:
                    dt[item2] = 1
                else:
                    dt[item2] += 1
    
    for key in list(dt.keys()):
        if len(key) <= 1:  
            del dt[key] 
    
    return dt

voxWords = normalizeString(voxUD)
voxWordsFlat =  wordDict(voxWords)        
voxWordsFlat = {k: v for k, v in sorted(voxWordsFlat.items(), reverse=True, key=lambda item: item[1])}
voxWordsFlat = {k: v for k, v in voxWordsFlat.items() if v > 2}
voxWordsFlat = list(voxWordsFlat.items())

ppWords = normalizeString(ppUD)
ppWordsFlat =  wordDict(ppWords)        
ppWordsFlat = {k: v for k, v in sorted(ppWordsFlat.items(), reverse=True, key=lambda item: item[1])}
ppWordsFlat = {k: v for k, v in ppWordsFlat.items() if v > 2}
ppWordsFlat = list(ppWordsFlat.items())

csWords = normalizeString(csUD)
csWordsFlat =  wordDict(csWords)        
csWordsFlat = {k: v for k, v in sorted(csWordsFlat.items(), reverse=True, key=lambda item: item[1])}
csWordsFlat = {k: v for k, v in csWordsFlat.items() if v > 2}
csWordsFlat = list(csWordsFlat.items())

psoeWords = normalizeString(psoeUD)
psoeWordsFlat =  wordDict(psoeWords)        
psoeWordsFlat = {k: v for k, v in sorted(psoeWordsFlat.items(), reverse=True, key=lambda item: item[1])}
psoeWordsFlat = {k: v for k, v in psoeWordsFlat.items() if v > 2}
psoeWordsFlat = list(psoeWordsFlat.items())

upWords = normalizeString(upUD)
upWordsFlat =  wordDict(upWords)        
upWordsFlat = {k: v for k, v in sorted(upWordsFlat.items(), reverse=True, key=lambda item: item[1])}
upWordsFlat = {k: v for k, v in upWordsFlat.items() if v > 2}
upWordsFlat = list(upWordsFlat.items())
#print(ppWordsFlat.keys(️️️)️️️)
#for item in list(ppWordsFlat.keys()):
    #print(len(item))
    
    
fList = [voxWordsFlat,ppWordsFlat,csWordsFlat,psoeWordsFlat,upWordsFlat]

flatTrans = []
for sublist in fList:
    for item in sublist:
        flatTrans.append(item[0])

diffWords = set(flatTrans)


parties = ["VOX", "PP", "CS", "PSOE", "UP"]

DannyTw = "soy de podemos!!!"

wordIdf = {}
for word in diffWords:
    wordIdf[word] = 0
    for index, party in enumerate(fList):
        for partyWord, num in party:
            if partyWord == word:
                wordIdf[word] += 1


voxWordNum = np.sum([value for i, value in fList[0]])
ppWordNum = np.sum([value for i, value in fList[1]])
csWordNum = np.sum([value for i, value in fList[2]])
psoeWordNum = np.sum([value for i, value in fList[3]])
upWordNum = np.sum([value for i, value in fList[4]])
WordPartiesSum = [voxWordNum, ppWordNum, csWordNum, psoeWordNum, upWordNum]

scores = {}
for index, party in enumerate(fList):
    scores[index] = 0
    
for i, value in DannyList:
    for index, party in enumerate(fList):
        for word, num in party:
            if i == word:
                scores[index] += value * np.log(len(fList)/wordIdf[word]) * num/WordPartiesSum[index]

print(scores)
list(scores.values())/np.sum(list(scores.values()))

ModuleNotFoundError: No module named 'TextBlob'

In [33]:
import regex
import emoji
from spacy.tokenizer import Tokenizer
from spacy.lang.es import Spanish

#Create spanish tokenizer.
nlp = Spanish()
tokenizer = Tokenizer(nlp.vocab)

name = "De donde has sacado la chapata!!!!!!"

#Function to remove emojis of a text.
def remove_emojis(text):
    string = ""
    data = regex.findall(r'\X', text)
    for word in data:
        for char in word:
            if not char in emoji.UNICODE_EMOJI:
                string = string + char
    return string

#Remove stop words of a text.
def removeStopwords(texto):
    texto = texto.lower()
    texto = re.sub('_', '', texto)
    texto = re.sub('@', '', texto)
    texto = re.sub(':', '', texto)
    texto = re.sub('/', '', texto)
    texto = re.sub('!', '', texto)
    texto = re.sub('_', '', texto)
    texto = re.sub('\.', '', texto)
    return texto

#Get important words of a text.
def getImportantWords(text):
    text_without_emojis = remove_emojis(text)
    text_cleaned = removeStopwords(text_without_emojis)
    tokenized_text = tokenizer(text_cleaned)
    text_important_words = [word for word in tokenized_text if word.is_stop == False]
    return text_important_words

#Return a dictionary with counter of each word in wor counted.
def dictWordCount(wordList, wDict):
    for word in wordList:
        if word in wDict:
            wDict[word] += 1
        else:
            wDict[word] = 1
    return wDict



[has, sacado, chapata]
